In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import time
import sys
import re
import math
import numpy  
import pandas as pd 
import os
import random

query_txt = input('    1.크롤링 할 영화의 제목을 입력하세요: ')
query_url = 'https://movie.naver.com'

cnt = int(input('    2.크롤링 할 리뷰건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("    3.파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")
print("-" *80)
print("    요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~~~")

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


options = Options()
options.binary_location= 'C:/Program Files/Google/Chrome/Application/chrome.exe'

driver = webdriver.Chrome('C:/py_temp/chromedriver_240/chromedriver.exe', chrome_options = options)

driver.get(query_url)
time.sleep(2)

element = driver.find_element_by_id("ipt_tx_srch")
element.send_keys(query_txt)
driver.find_element_by_xpath("""//*[@id="jSearchArea"]/div/button""").click()
driver.find_element_by_xpath("""//*[@id="old_content"]/ul[2]/li/dl/dt/a""").click()
        
driver.find_element_by_link_text("평점").click()


driver.switch_to.frame('pointAfterListIframe')

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result= soup.find('div', class_='score_total').find('strong','total').find_all('em')
result2 = result[0].get_text()

print("=" *80)
result3 = result2.replace(",","")
result4 = re.search("\d+",result3)
search_cnt = int(result4.group())

score2=[]
review2=[]
writer2=[]
wdate2=[]
gogam2=[]
g_gogam2=[]
b_gogam2=[]
dwlist2=[]

count = 0         
click_count = 1  

while ( click_count <= page_cnt )  :
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            
            score_result = soup.find('div', class_='score_result').find('ul')
            slist = score_result.find_all('li')

            for li in slist:

                count += 1
                
                f = open(ff_name, 'a',encoding='UTF-8')
            
                print("\n")
                print("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count))
                score = li.find('div', class_='star_score').find('em').get_text()
                print("1.별점:","*"*int(score),": ", score)
                score2.append(score)
                f.write("\n")
                f.write("총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다==============" %(cnt,count) + "\n")
                f.write("1.별점:"+score + "\n")
            
                review = li.find('div', class_='score_reple').find('p').get_text()
                review = review.strip()
                print("2.리뷰내용:",review)
                f.write("2.리뷰내용:" + review + "\n")
                review2.append(review)
         
                dwlist = li.find('div',class_='score_reple').find_all('em')
                writer = dwlist[0].find('span').get_text()
                print("3.작성자:",writer)
                f.write("3.작성자:" + writer + "\n")
                writer2.append(writer)
                
                wdate = dwlist[1].text
                print('4.작성일자:',wdate)
                f.write("4.작성일자:" + wdate + "\n")
                wdate2.append(wdate)
            
                gogam = li.find('div', class_='btn_area').find_all('strong')
                g_gogam = gogam[0].text
                print('5.공감:',g_gogam)
                f.write("5.공감:" + g_gogam + "\n")
                g_gogam2.append(g_gogam)
                
                b_gogam = gogam[1].text
                print('6.비공감:', b_gogam)
                f.write("6.비공감:" + b_gogam + "\n")
                b_gogam2.append(b_gogam)
                print("\n")

                if count == cnt :
                    break
            
            time.sleep(random.randrange(1,2))  

            click_count += 1
            
            if click_count > page_cnt :
                break
            else :
                driver.find_element_by_link_text("%s" %click_count).click()

            time.sleep(random.randrange(1,2))


naver_movie = pd.DataFrame()
naver_movie['별점(평점)']=score2
naver_movie['리뷰내용']=review2
naver_movie['작성자']=writer2
naver_movie['작성일자']=wdate2
naver_movie['공감횟수']=g_gogam2
naver_movie['비공감횟수']=b_gogam2

naver_movie.to_csv(fc_name,encoding="utf-8-sig",index=True)

naver_movie.to_excel(fx_name ,index=True)

driver.close( )

    1.크롤링 할 영화의 제목을 입력하세요: 김씨 표류기
    2.크롤링 할 리뷰건수는 몇건입니까?: 30
    3.파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\\data\\
--------------------------------------------------------------------------------
    요청하신 데이터를 수집 중이오니 잠시만 기다려 주세요~~~~


<ipython-input-8-cd5fce7cec48>:37: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:/py_temp/chromedriver_240/chromedriver.exe', chrome_options = options)




총 30 건 중 1 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: [김씨표류기]를 재개봉해주세요.지지해주신분은 좋아요를 눌러주세요.
3.작성자: zhan****
4.작성일자: 2016.04.04 11:35
5.공감: 3635
6.비공감: 41




총 30 건 중 2 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 와...방금 봤는데 진짜 영화 잘만들었다...이런 영화들이 막 500만넘고 그래야되는데...요즘보면 무슨 은밀하게위대하게같은 쓰레기영화가 독점해서ㅉㅉㅉ
3.작성자: gust****
4.작성일자: 2013.06.17 04:33
5.공감: 2012
6.비공감: 180




총 30 건 중 3 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: 가장섬세한 한국영화 ,, 감독 여자인줄 알았다 너무좋다 감독이 여자든 남자든
3.작성자: 문슬레이(jkm6****)
4.작성일자: 2013.04.10 21:13
5.공감: 1289
6.비공감: 113




총 30 건 중 4 번째 리뷰 데이터를 수집합니다==============
1.별점: ********** :  10
2.리뷰내용: OO 모르면서 1점주지마 이게 그냥 겨우 시시한 개그영화로 보이면 영화 보는눈이 전혀 없는거다 괜히 이해못하니까
3.작성자: rkdc****
4.작성일자: 2013.05.27 18:24
5.공감: 1057
6.비공감: 63




총 30 건 중 5 번째 리뷰 데이터를 수집합니다==============
1.별점: ******** :  8
2.리뷰내용: 심오하면서 재치있으면서 감성적이다. 한국에서 좀처럼 찾아보기 힘든 감성으로 참신하고 신선하다. 후반이 예상 가능하다는 것 말고는..
3.작성자: 산토키(chur****)
4.작성일자: 2013.05.30 23:07
5.공감: 994
6.비공감: